In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import polars as pl
from tqdm.notebook import tqdm

In [2]:
# Check data info
data_info = pd.read_excel('/opt/nilm-shared-data/nilm_device_detection/ICTA2024_dataset/data_information/data_information.xlsx', sheet_name='data_info')
data_patch = pd.read_excel('/opt/nilm-shared-data/nilm_device_detection/ICTA2024_dataset/data_information/data_information.xlsx', sheet_name='data_patch')

In [3]:
data_info.head()

,index,combination,data1,data2,data3,data4
0,0,[0],20240121_data_Nodevice_1.xlsx,20240121_data_Nodevice_2.xlsx,20240121_data_Nodevice_3.xlsx,20240121_data_Nodevice_4.xlsx
1,1,[1],20240113_data_den_1.xlsx,20240114_data_den_2.xlsx,20240115_data_den_3.xlsx,20240121_data_den_4.xlsx
2,2,[2],20240113_data_quat_1.xlsx,20240114_data_quat_2.xlsx,20240115_data_quat_3.xlsx,20240121_data_quat_4.xlsx
3,3,"[1, 2]",20240114_data_den_quat_1.xlsx,20240114_data_den_quat_2.xlsx,20240115_data_den_quat_3.xlsx,20240121_data_den_quat_4.xlsx
4,4,[3],20240113_data_maysay_1.xlsx,20240114_data_maysay_2.xlsx,20240115_data_maysay_3.xlsx,20240121_data_maysay_4.xlsx


In [4]:
data_patch.head()

,STT,Combinations,Files,Start,End
0,0,No device,"1,2,3,4",NaN,NaN
1,1,[1],"2,3,4","0,0,0","0,80000,0"
2,2,[2],"1,2,3,4",NaN,NaN
3,3,"[1, 2]","1,2,3,4",NaN,NaN
4,4,[3],"1,2,3,4",NaN,NaN


In [5]:
len(data_info), len(data_patch)

(128, 128)

In [7]:
data_info.iloc[73]

index                                               73
combination                                  [1, 4, 7]
data1          20240116_data_den_maylockk_sacmt_1.xlsx
data2          20240118_data_den_maylockk_sacmt_2.xlsx
data3          20240120_data_den_maylockk_sacmt_3.xlsx
data4          20240122_data_den_maylockk_sacmt_4.xlsx
Name: 73, dtype: object

In [6]:
base_path = "/opt/nilm-shared-data/nilm_device_detection/ICTA2024_dataset/raw_data"
save_path = "/opt/nilm-shared-data/nilm_device_detection/ICTA2024_dataset/fix_raw_data"
file_col_names = ["data1", "data2", "data3", "data4"]
for comb_index in tqdm([97, 73, 102, 113], desc="Processing data"):
    # Resetting fix_df
    fix_df = None
    # Getting processed files
    file_nos = data_patch.iloc[comb_index]["Files"]
    file_nos = file_nos.replace(" ", "")
    file_nos = file_nos.split(",")
    file_nos = [int(file_no) for file_no in file_nos]
    # Getting start and end times
    start_times = data_patch.iloc[comb_index]["Start"]
    end_times = data_patch.iloc[comb_index]["End"]
    print(f"[+] Comb index: {comb_index}, File Nos: {file_nos}, start times: {start_times}, end times: {end_times}")
    # Processing data
    if start_times is not np.nan and end_times is not np.nan:
        # Getting start and end index
        start_times = start_times.replace(" ", "")
        start_times = start_times.split(",")
        start_times = [int(start_time) for start_time in start_times]
        end_times = end_times.replace(" ", "")
        end_times = end_times.split(",")
        end_times = [int(end_time) for end_time in end_times]
        # Cutting data
        for file_no, start, end in zip(file_nos, start_times, end_times): 
            file_name = data_info.iloc[comb_index][file_col_names[file_no-1]]
            df = pd.read_excel(f"{base_path}/{file_name}")
            if end == 0:
                fix_df = df.iloc[start:]
            else:
                fix_df = df.iloc[start:end]
            fix_df = fix_df.reset_index(drop=True)
            file_name = file_name.replace(".xlsx", ".csv")
            fix_df.to_csv(f"{save_path}/{file_name}", index=False)
            print(f"[-] File: {file_name} saved with shape: {fix_df.shape}")
    else:
        for file_no in file_nos:
            file_name = data_info.iloc[comb_index][file_col_names[file_no-1]]
            fix_df = pd.read_excel(f"{base_path}/{file_name}")
            file_name = file_name.replace(".xlsx", ".csv")
            fix_df.to_csv(f"{save_path}/{file_name}", index=False)
            print(f"[-] File: {file_name} saved with shape: {fix_df.shape}")

Processing data:   0%|          | 0/4 [00:00<?, ?it/s]

[+] Comb index: 97, File Nos: [1, 2, 3, 4], start times: 0,0,0,0, end times: 160000,160000,125000,0
[-] File: 20240116_data_den_amnuoc_sacmt_1.csv saved with shape: (160000, 4)
[-] File: 20240118_data_den_amnuoc_sacmt_2.csv saved with shape: (160000, 4)
[-] File: 20240120_data_den_amnuoc_sacmt_3.csv saved with shape: (125000, 4)
[-] File: 20240122_data_den_amnuoc_sacmt_4.csv saved with shape: (180000, 4)
[+] Comb index: 73, File Nos: [1, 4], start times: nan, end times: nan
[-] File: 20240116_data_den_maylockk_sacmt_1.csv saved with shape: (180000, 4)
[-] File: 20240122_data_den_maylockk_sacmt_4.csv saved with shape: (180000, 4)
[+] Comb index: 102, File Nos: [1, 2, 3, 4], start times: 0,0,0,0, end times: 160000,0,0,0
[-] File: 20240117_data_quat_maysay_amnuoc_sacmt_1.csv saved with shape: (160000, 4)
[-] File: 20240119_data_quat_maysay_amnuoc_sacmt_2.csv saved with shape: (180000, 4)
[-] File: 20240120_data_quat_maysay_amnuoc_sacmt_3.csv saved with shape: (180000, 4)
[-] File: 2024012